In [2]:
import pandas as pd
import numpy as np
import talib
import pandas_ta as ta
from backtester import BackTester
import matplotlib.pyplot as plt

def process_data(data):
    data['ATR'] = ta.atr(data['high'], data['low'], data['close'], length=14)
    data['SMA_20'] = ta.sma(data['close'], length=20)
    data['SMA_50'] = ta.sma(data['close'], length=50)
    data['RSI'] = ta.rsi(data['close'], length=14)
    data['Volume_SMA'] = ta.sma(data['volume'], length=20)
    data['Volume_Ratio'] = data['volume'] / data['Volume_SMA']
    data['Price_Change'] = data['close'].pct_change()
    data['Price_Volatility'] = data['Price_Change'].rolling(window=14).std()
    return data

def strat(data):
    data['trade_type'] = "HOLD"
    data['signals'] = 0
    position = 0
    lookback_period = 14
    for i in range(lookback_period, len(data)):
        if data['SMA_20'].iloc[i] > data['SMA_50'].iloc[i] and position != 1:
            data.loc[i, 'signals'] = 1
            position = 1
            data.loc[i, 'trade_type'] = "LONG"
        elif data['SMA_20'].iloc[i] < data['SMA_50'].iloc[i] and position != 0:
            data.loc[i, 'signals'] = -1
            position = 0
            data.loc[i, 'trade_type'] = "CLOSE_LONG"
    return data

def plot_custom_graphs(bt):
    # Trade graph
    trades = bt.get_trade_data()
    if trades is not None and not trades.empty:
        plt.figure(figsize=(12, 6))
        plt.plot(trades['timestamp'], trades['price'], label='Price')
        plt.scatter(trades[trades['trade_type'] == 'LONG']['timestamp'], 
                    trades[trades['trade_type'] == 'LONG']['price'], 
                    label='Buy', color='green', marker='^')
        plt.scatter(trades[trades['trade_type'] == 'CLOSE_LONG']['timestamp'], 
                    trades[trades['trade_type'] == 'CLOSE_LONG']['price'], 
                    label='Sell', color='red', marker='v')
        plt.title('Trade Graph')
        plt.legend()
        plt.grid()
        plt.savefig("trade_graph.png")
        plt.show()

    # PnL graph
    pnl_data = bt.get_pnl_data()
    if pnl_data is not None and not pnl_data.empty:
        plt.figure(figsize=(12, 6))
        plt.plot(pnl_data['timestamp'], pnl_data['pnl'], label='PnL', color='blue')
        plt.title('PnL Over Time')
        plt.grid()
        plt.savefig("pnl_graph.png")
        plt.show()

def main():
    data = pd.read_csv("BTC_2019_2023_1d.csv")
    processed_data = process_data(data)
    result_data = strat(processed_data)

    print("Signals in generated data:", result_data['signals'].unique())
    print("Number of non-zero signals:", (result_data['signals'] != 0).sum())

    result_data.to_csv("final_data.csv", index=False)

    bt = BackTester("BTC", signal_data_path="final_data.csv", master_file_path="final_data.csv", compound_flag=1)
    bt.get_trades(1000)

    stats = bt.get_statistics()
    if stats:
        for key, val in stats.items():
            print(f"{key}: {val}")
    else:
        print("No statistics available. Did your strategy make any trades?")

    try:
        bt.make_trade_graph()
        bt.make_pnl_graph()
        plt.show()
    except:
        print("Built-in graph functions failed. Using custom matplotlib plotting...")
        plot_custom_graphs(bt)

if __name__ == "__main__":
    main()


Signals in generated data: [ 0  1 -1]
Number of non-zero signals: 33
Total Trades: 16
Leverage Applied: 1
Winning Trades: 5
Losing Trades: 11
No. of Long Trades: 16
No. of Short Trades: 0
Benchmark Return(%): 325.632937277405
Benchmark Return(on $1000): 3256.32937277405
Win Rate: 31.25
Winning Streak: 2
Losing Streak: 7
Gross Profit: 1790.252125076904
Net Profit: 1718.6376785381472
Average Profit: 107.4148549086342
Maximum Drawdown(%): 49.50179668698343
Average Drawdown(%): 21.109775685032496
Largest Win: 3760.5971265517123
Average Win: 972.1609540927184
Largest Loss: -722.4555948842293
Average Loss: -285.6515538114041
Maximum Holding Time: 197 days 00:00:00
Average Holding Time: 45 days 18:00:00
Maximum Adverse Excursion: None
Average Adverse Excursion: None
Sharpe Ratio: 3.9252630636404047
Sortino Ratio: None
